In [1]:
!pip install elasticsearch==7.9.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 KB 8.6 MB/s eta 0:00:00


In [2]:
%%bash

wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-7.9.2/
shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512 

elasticsearch-oss-7.9.2-linux-x86_64.tar.gz: OK


In [3]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

In [4]:
import time

In [5]:
# Sleep for few seconds to let the instance start.
time.sleep(20)

In [6]:
%%bash

ps -ef | grep elasticsearch

root         603     601  0 14:26 ?        00:00:00 sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch
daemon       604     603 99 14:26 ?        00:00:21 /content/elasticsearch-7.9.2/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/elasticsearch-2116656603415934780 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:filecou

In [8]:
%%bash

curl -sX GET "localhost:9200/"

{
  "name" : "5269925da9ed",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "7EKAbe7RShWX19vh6DlKGA",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [19]:
csv_file = '/content/Video_Games_Sales_as_at_22_Dec_2016.csv'

In [18]:
import pandas as pd

In [12]:
df = pd.read_csv(csv_file)

In [13]:
df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df = df.dropna(axis=0,subset=['Name','Global_Sales', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Publisher'])

In [15]:
df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
len(df), len(df.columns)

(16663, 16)

In [20]:
records = df.to_dict(orient="records")
records[0]

{'Name': 'Wii Sports',
 'Platform': 'Wii',
 'Year_of_Release': 2006.0,
 'Genre': 'Sports',
 'Publisher': 'Nintendo',
 'NA_Sales': 41.36,
 'EU_Sales': 28.96,
 'JP_Sales': 3.77,
 'Other_Sales': 8.45,
 'Global_Sales': 82.53,
 'Critic_Score': 76.0,
 'Critic_Count': 51.0,
 'User_Score': '8',
 'User_Count': 322.0,
 'Developer': 'Nintendo',
 'Rating': 'E'}

In [21]:
from elasticsearch import Elasticsearch

In [22]:
ES_NODES = "http://localhost:9200"

def prepare_es_data(index, doc_type, df):
  records = df.to_dict(orient="records")
  es_data = []
  for idx, record in enumerate(records):
    meta_dict = {
          "index": {
              "_index": index, 
              "_type": doc_type, 
              "_id": idx
          }
      }
    es_data.append(meta_dict)
    es_data.append(record)

  return es_data

def index_es_data(index, es_data):
  es_client = Elasticsearch(hosts = [ES_NODES])
  if es_client.indices.exists(index):
      print("deleting the '{}' index.".format(index))
      res = es_client.indices.delete(index=index)
      print("Response from server: {}".format(res))

  print("creating the '{}' index.".format(index))
  res = es_client.indices.create(index=index)
  print("Response from server: {}".format(res))

  print("bulk index the data")
  res = es_client.bulk(index=index, body=es_data, refresh = True)
  print("Errors: {}, Num of records indexed: {}".format(res["errors"], len(res["items"])))

In [33]:
datamod = prepare_es_data(index="games", doc_type="newgame", df=df)

In [34]:
datamod

[{'index': {'_index': 'games', '_type': 'newgame', '_id': 0}},
 {'Name': 'Wii Sports',
  'Platform': 'Wii',
  'Year_of_Release': 2006.0,
  'Genre': 'Sports',
  'Publisher': 'Nintendo',
  'NA_Sales': 41.36,
  'EU_Sales': 28.96,
  'JP_Sales': 3.77,
  'Other_Sales': 8.45,
  'Global_Sales': 82.53,
  'Critic_Score': 76.0,
  'Critic_Count': 51.0,
  'User_Score': '8',
  'User_Count': 322.0,
  'Developer': 'Nintendo',
  'Rating': 'E'},
 {'index': {'_index': 'games', '_type': 'newgame', '_id': 1}},
 {'Name': 'Super Mario Bros.',
  'Platform': 'NES',
  'Year_of_Release': 1985.0,
  'Genre': 'Platform',
  'Publisher': 'Nintendo',
  'NA_Sales': 29.08,
  'EU_Sales': 3.58,
  'JP_Sales': 6.81,
  'Other_Sales': 0.77,
  'Global_Sales': 40.24,
  'Critic_Score': nan,
  'Critic_Count': nan,
  'User_Score': nan,
  'User_Count': nan,
  'Developer': nan,
  'Rating': nan},
 {'index': {'_index': 'games', '_type': 'newgame', '_id': 2}},
 {'Name': 'Mario Kart Wii',
  'Platform': 'Wii',
  'Year_of_Release': 2008.0

In [35]:
index_es_data(index="games", es_data=datamod)

creating the 'games' index.
Response from server: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'games'}
bulk index the data


/usr/local/lib/python3.9/dist-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


Errors: True, Num of records indexed: 16663


In [48]:
es_client = Elasticsearch(hosts = [ES_NODES])

In [37]:
query = {
"size": 1,
"sort": [
{"Global_Sales": {"order": "desc"}}
],
"query": {
"match_all": {}
}
}
# Execute the search query
response = es_client.search(index="games", body=query)
# Extract the name of the best selling game from the response
best_selling_game = response["hits"]["hits"][0]["_source"]["Name"]
print("The best selling game overall is:", best_selling_game)

The best selling game overall is: Wii Sports


In [39]:
query = {
"size": 0,
"aggs": {
"genres": {
"terms": {
"field": "Genre.keyword",
"size": 100
},
"aggs": {
"total_sales": {
"sum": {
"field": "Global_Sales"
}
}
}
}
},
"sort": [
{"Global_Sales": {"order": "desc"}}
]
}
# Execute the search query
response = es_client.search(index="games", body=query)
# Extract the name of the best selling genre from the response
best_selling_genre = response["aggregations"]["genres"]["buckets"][0]["key"]
print("The best selling genre overall is:", best_selling_genre)


The best selling genre overall is: Action


In [40]:
# Define the search query
query = {
"size": 0,
"aggs": {
"publishers": {
"terms": {
"field": "Publisher.keyword",
"size": 100
},
"aggs": {
"total_sales": {
"sum": {
"field": "Global_Sales"
}
}
}
}
},
"sort": [
{"Global_Sales": {"order": "desc"}}
]
}
# Execute the search query
response = es_client.search(index="games", body=query)
# Extract the name of the publisher with the highest global sales from the response
best_publisher = response["aggregations"]["publishers"]["buckets"][0]["key"]
print("The publisher with the highest number of global sales is:", best_publisher)

The publisher with the highest number of global sales is: Electronic Arts


In [44]:
# Define the search query for the average North American sales
query_na_sales = {
"size": 0,
"aggs": {
"average_na_sales": {
"avg": {
"field": "NA_Sales"
}
}
}
}
# Define the search query for the average global sales
query_global_sales = {
"size": 0,
"aggs": {
"average_global_sales": {
"avg": {
"field": "Global_Sales"
}
}
}
}
# Execute the search query for the average North American sales
response_na_sales = es_client.search(index="games", body=query_na_sales)
# Execute the search query for the average global sales
response_global_sales = es_client.search(index="games", body=query_global_sales)
# Extract the average North American sales and average global sales from the responses
average_na_sales = response_na_sales["aggregations"]["average_na_sales"]["value"]
average_global_sales =response_global_sales["aggregations"]["average_global_sales"]["value"]
# Print the results
print("The average sales in North America is:", average_na_sales)
print("The average global sales is:", average_global_sales)


The average sales in North America is: 0.3944835162544862
The average global sales is: 0.7775897426384709


In [46]:
# Define the search query to retrieve the top 5 games with the most global sales
query = {
"query": {
"match_all": {}
},
"sort": [
{"Global_Sales": {"order": "desc"}}
],
"size": 5
}
# Execute the search query to retrieve the top 5 games with the most global sales
response = es_client.search(index="games", body=query)
# Print the name and genre of each game with the most global sales
for hit in response["hits"]["hits"]:
  game_title = hit["_source"]["Name"]
  game_genre = hit["_source"]["Genre"]
  print(f"{game_title} - {game_genre}")

Wii Sports - Sports
Mario Kart Wii - Racing
Wii Sports Resort - Sports
New Super Mario Bros. - Platform
Wii Play - Misc


In [47]:
# Define the search query to retrieve all the games that have a genre of "Action" or "Fighting"
query = {
"query": {
"bool": {
"should": [
{"match": {"Genre": "Action"}},
{"match": {"Genre": "Fighting"}}
]
}
}
}
# Execute the search query to retrieve all the games that have a genre of "Action" or "Fighting"
response = es_client.search(index="games", body=query)
# Print the titles of all the games that have a genre of "Action" or "Fighting"
print("Games categorized as 'Action' or 'Fighting':")
for hit in response["hits"]["hits"]:
  game_title = hit["_source"]["Name"]
  print(game_title)


Games categorized as 'Action' or 'Fighting':
Super Smash Bros. Brawl
Tekken 3
Super Smash Bros. Melee
Tekken 2
Street Fighter IV
Tekken Tag Tournament
Tekken 5
Tekken 4
Dragon Ball Z: Budokai
Dragon Ball Z: Budokai Tenkaichi 3
